In [4]:
import torch; torch.set_grad_enabled(False)
from transformers import AutoTokenizer, AutoModelForCausalLM
from utils import load_orig_ds_txt, tokenize, get_correct_probs, probs_to_colors, vis_tokens
from IPython.display import HTML

model_a_name = "roneneldan/TinyStories-1M"
model_b_name = "roneneldan/TinyStories-3M"
tokenizer = AutoTokenizer.from_pretrained("roneneldan/TinyStories-1M")
model_a = AutoModelForCausalLM.from_pretrained(model_a_name)
model_b = AutoModelForCausalLM.from_pretrained(model_b_name)
ds_txt = load_orig_ds_txt("validation[:10]")
ds_tok = [tokenize(tokenizer, txt) for txt in ds_txt]

/Users/jett/Documents/jett/tinyevals/.venv/lib/python3.10/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
def to_tok_prob_str(tok, prob_a, prob_b):
    tok_str = tokenizer.decode(tok).replace(" ", "&nbsp;").replace("\n", r"\n")
    prob_a_str = f"{prob_a:.2%}"
    prob_b_str = f"{prob_b:.2%}"
    return f"{prob_a_str:>6} → {prob_b_str:>6} |{tok_str}|"

def compare_models(tokens):
    probs_a = get_correct_probs(model_a, tokens)
    probs_b = get_correct_probs(model_b, tokens)
    probs_above = torch.maximum(probs_b - probs_a, torch.zeros_like(probs_a))
    colors = probs_to_colors(probs_above)
    hover_datas = [{}]
    for i in range(1, len(tokens)):
        tok = tokens[i]
        prob_a = probs_a[i-1]
        prob_b = probs_b[i-1]
        hover_datas.append(dict(next=to_tok_prob_str(tok, prob_a, prob_b)))
    display(HTML(vis_tokens(tokenizer, tokens, colors, hover_datas)))

In [11]:
# try a few different examples
# the JS hovering currently only works for one example at a time
i = 4
compare_models(ds_tok[i])